## REQUIREMENTS

In [ ]:
# !pip install efficientnet_pytorch

import torch
import torch.nn as nn
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.io import read_image
from efficientnet_pytorch import EfficientNet
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import random
import os
import numpy as np

seed = 123
random.seed(seed)
np.random.seed(seed)
_ = torch.manual_seed(seed)
_ = torch.cuda.manual_seed(seed)

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=796c52704181e7bf021bf3254fff46bdc3ab2349ecf3622071f5d2a7c2347b01
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


## Defining our Neural Network

In [11]:
# Definim la classe MangoNet que hereta d'EfficientNet
class MangoNet(nn.Module):
    def __init__(self, num_channels=11*(33+1),drop=0.5):
        # Carrega EfficientNet-B1 preentrenat
        super(MangoNet, self).__init__()
        # Carrega el model EfficientNet-B1 amb pesos preentrenats
        efficient_net = EfficientNet.from_pretrained('efficientnet-b2')

        # Obté el nombre d'entrades de l'última capa fully connected original
        self.features = nn.Sequential(
            efficient_net._conv_stem,
            efficient_net._bn0,
            *efficient_net._blocks,
            efficient_net._conv_head,
            efficient_net._bn1,
        )
        in_features = efficient_net._fc.in_features

        # Substitueix l'última capa amb una nova capa Conv2d per tenir la sortida amb els canals desitjats
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=drop, inplace=True),
            nn.Linear(in_features, num_channels),
        )


    def forward(self, x):
        # Passa l'entrada a través de les capes de features d'EfficientNet
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        # Aplica la nostra capa Conv2d personalitzada per obtenir la sortida desitjada
        x = self.classifier(x)
        x = torch.reshape(x, [-1,34,11])
        return x


## Importing the dataest

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!unzip '/content/drive/MyDrive/datathon-fme-mango.zip'

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: archive/images/images/88_49713919_67042010-37_.jpg  
  inflating: archive/images/images/88_49713920_67042010-56_.jpg  
  inflating: archive/images/images/88_49713921_67042910-02_.jpg  
  inflating: archive/images/images/88_49713922_67042910-56_.jpg  
  inflating: archive/images/images/88_49713923_67042910-99_.jpg  
  inflating: archive/images/images/88_49713924_67042913-TC_.jpg  
  inflating: archive/images/images/88_49713925_67042913-TO_.jpg  
  inflating: archive/images/images/88_49713928_67050465-01_B.jpg  
  inflating: archive/images/images/88_49713929_67050465-TN_B.jpg  
  inflating: archive/images/images/88_49713930_67050466-99_.jpg  
  inflating: archive/images/images/88_49713934_67051010-95_.jpg  
  inflating: archive/images/images/88_49713935_67051011-08_.jpg  
  inflating: archive/images/images/88_49713936_67053266-56_B.jpg  
  inflating: archive/images/images/88_49713937_67053685-08_B.jpg  
  inflating

### Definint the data loader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        """
        Inicialitza el conjunt de dades personalitzat.

        Args:
            csv_file (str): El camí al fitxer CSV amb les etiquetes.
            image_dir (str): El directori que conté les imatges.
            transform (callable, optional): Transformacions a aplicar a les imatges.
        """
        self.img_labels = pd.read_csv(csv_file)  # Carrega el CSV amb les etiquetes
        self.img_labels['filename'] = self.img_labels['filename'].astype(str)
        self.img_labels = self.img_labels.drop(columns=['cod_modelo_color'])
        self.img_dir = image_dir  # Directori de les imatges
        self.transform = transform  # Transformacions (per exemple, redimensionar, normalitzar)

        # Preparem les etiquetes
        self.label_columns = self.img_labels.columns.difference(['filename'])

        # Codificació de les etiquetes si són categòriques
        self.label_encoders = {}
        for col in self.label_columns:
            le = LabelEncoder()
            self.img_labels[col] = le.fit_transform(self.img_labels[col])  # Codifiquem les etiquetes
            self.label_encoders[col] = le  # Guardem l'encoder per a futures prediccions


    def __len__(self):
        """Retorna la mida total del conjunt de dades."""
        return len(self.img_labels)

    def __getitem__(self, index):
        """
        Obté una imatge i la seva etiqueta corresponent.

        Args:
            index (int): Índex de la imatge i etiqueta a recuperar.

        Returns:
            tuple: (imatge, etiqueta) on l'imatge és un tensor i l'etiqueta és un tensor.
        """
        # Obtenir el camí de la imatge
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[index, 0])  # 'filename' és a la primera columna
        image = read_image(img_path)  # Obrim l'imatge i la convertim a RGB



        # Aplica transformacions si n'hi ha
        if self.transform:
            image = self.transform(image)


        label = self.img_labels.iloc[index,1:]
        label = label.values.astype('int')
        image = image.type(torch.float32)
        return image, label

## Training costumization

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


# Definir el teu model
model = MangoNet(num_channels=11 * (33 + 1), drop=0.5)
model = model.to('cuda')
# Defineix la funció de pèrdua (per exemple, CrossEntropyLoss per classificació)
criterion = nn.CrossEntropyLoss()

# Definir diferents learning rates per a les capes preentrenades i la nova capa
optimizer = optim.Adam([
    {'params': model.features.parameters(), 'lr': 1e-4},  # LR més petit per a les capes preentrenades
    {'params': model.classifier.parameters(), 'lr': 1e-3}  # LR més gran per a la nova capa
])

# Configuració de l'entrenament


num_epochs = 1
dataset = CustomDataset(csv_file='/content/drive/MyDrive/clean_data.csv', image_dir='/content/archive/images/images', transform=None)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Entrenament
for epoch in range(num_epochs):
    model.train()  # Mode entrenament
    running_loss = 0.0
    it = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Envia les dades a la GPU si en tens una

        # Zero gradients
        optimizer.zero_grad()
        it += 1
        # Forward pass
        try:
          outputs = model(inputs)
        except Exception as ex:
          print(ex)
          breakpoint()
        loss = criterion(outputs, labels)
        print(loss.item())

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Època {epoch + 1}/{num_epochs}, Pèrdua: {running_loss / len(train_loader)}")

torch.save(model, "namemodel.pt")


Loaded pretrained weights for efficientnet-b1
4.92567253112793
1.8264293670654297
1.0971027612686157
1.2289905548095703
1.3202675580978394
1.3278244733810425
1.3378056287765503
1.2777177095413208
1.2343087196350098
1.050007939338684
1.1341971158981323
1.1074469089508057
1.1363857984542847
1.0133472681045532
1.0084476470947266
1.0826798677444458
1.066153645515442
0.8579511046409607
0.8975335955619812
0.9165595769882202
0.9968633055686951
0.9431298971176147
0.8825764060020447
0.9313268065452576
0.9302706122398376
0.8769850134849548
0.8759416937828064
0.9203828573226929
0.8766920566558838
0.9258091449737549
0.8668561577796936
0.857090175151825
0.8217349648475647
0.752528965473175
0.6973828673362732
0.7544535994529724
0.7197142839431763
0.7534515261650085
0.7572476863861084
0.7982320189476013
0.789896547794342
0.7677843570709229
0.7407808303833008
0.8235841989517212
0.8091855645179749
0.6795288324356079
0.706045925617218
0.7874622344970703
0.7777843475341797
0.7541196346282959
0.7428664565

### Loading trained parameters

In [12]:
model_guillem = MangoNet(num_channels=11*(33+1),drop=0.5)
model_guillem.load_state_dict(torch.load('/content/drive/MyDrive/params4b2'))

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth
100%|██████████| 35.1M/35.1M [00:00<00:00, 98.4MB/s]
<ipython-input-12-e549e1668c76>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use c

Loaded pretrained weights for efficientnet-b2


<All keys matched successfully>

In [ ]:
class TestLoader(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        """
        Inicialitza el conjunt de dades personalitzat.

        Args:
            csv_file (str): El camí al fitxer CSV amb les etiquetes.
            image_dir (str): El directori que conté les imatges.
            transform (callable, optional): Transformacions a aplicar a les imatges.
        """
        self.img_labels = pd.read_csv(csv_file)[:4096]  # Carrega el CSV amb les etiquetes
        self.img_labels['filename'] = self.img_labels['filename'].astype(str)
        self.img_labels = self.img_labels.drop(columns=['cod_modelo_color'])
        self.img_dir = image_dir  # Directori de les imatges
        self.transform = transform  # Transformacions (per exemple, redimensionar, normalitzar)

        # Preparem les etiquetes
        self.label_columns = self.img_labels.columns.difference(['filename'])

        # Codificació de les etiquetes si són categòriques
        self.label_encoders = {}
        for col in self.label_columns:
            le = LabelEncoder()
            self.img_labels[col] = le.fit_transform(self.img_labels[col])  # Codifiquem les etiquetes
            self.label_encoders[col] = le  # Guardem l'encoder per a futures prediccions


    def __len__(self):
        """Retorna la mida total del conjunt de dades."""
        return len(self.img_labels)

    def __getitem__(self, index):
        """
        Obté una imatge i la seva etiqueta corresponent.

        Args:
            index (int): Índex de la imatge i etiqueta a recuperar.

        Returns:
            tuple: (imatge, etiqueta) on l'imatge és un tensor i l'etiqueta és un tensor.
        """
        # Obtenir el camí de la imatge
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[index, 0])  # 'filename' és a la primera columna
        image = read_image(img_path)  # Obrim l'imatge i la convertim a RGB



        # Aplica transformacions si n'hi ha
        if self.transform:
            image = self.transform(image)


        label = self.img_labels.iloc[index,1:]
        label = label.values.astype('int')
        image = image.type(torch.float32)
        return image, self.img_labels['filename'].loc[index]

In [15]:
dataset = CustomDataset(csv_file='/content/drive/MyDrive/clean_data.csv', image_dir='/content/archive/images/images', transform=None)

### Computing predictions for the test dataset

#### Without batches

In [ ]:
# Load the CSV containing image paths
test_data = pd.read_csv('/content/test_data (1).csv').drop_duplicates(subset=['cod_modelo_color'])
image_dir = '/content/archive/images/images'
model_guillem = model_guillem.to(device)  # Ensure model is on the correct device
model_guillem.eval()
ID=0

rows_output = pd.DataFrame(columns=['test_id', 'des_value'])

# Iterate through image paths in the CSV
for index, row in test_data.iterrows():
    filename = row['des_filename']  # Assuming 'filename' column contains image paths
    img_path = os.path.join(image_dir, filename)


    if not os.path.exists(img_path) or os.path.getsize(img_path) == 0: continue
    # Load and preprocess the image
    image = read_image(img_path)
    image = image.type(torch.float32)
    image = image.unsqueeze(0)  # Add batch dimension
    image = image.to(device)    # Move to device


    # Make prediction
    pred = model_guillem(image)
    pred_np = pred.cpu().detach().numpy()
    pred_labels = np.argmax(pred_np, axis=1)

    # Get labels and encoders from the dataset used for training (dataset)
    tags = dataset.label_encoders.keys()
    d = {}

    for i, t in enumerate(tags):
        row_id = filename[:11]  # Extract row ID from filename
        encoder = dataset.label_encoders[t]
        label_for_tag = pred_labels[0, i]
        try: d[t] = encoder.inverse_transform([label_for_tag])[0]
        except: d[t] = 'INVALID'

        row_data = {'test_id': row_id + '_' + t, 'des_value': d[t]}
        rows_output = pd.concat([rows_output, pd.DataFrame([row_data])], ignore_index=True)
# Save predictions to CSV
rows_output.to_csv('MangoLoco-RandomGuess.csv', index=False)

#### With batches

In [ ]:
# Configuració
batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Carrega el CSV i inicialitza el model
test_data = pd.read_csv('/content/archive/test_data.csv').drop_duplicates(subset=['cod_modelo_color'])
image_dir = '/content/archive/images/images'
model_guillem = model_guillem.to(device)
model_guillem.eval()

rows_output = pd.DataFrame(columns=['test_id', 'des_value'])

# Variables per acumular imatges i noms d'arxiu
image_batch = []
filename_batch = []

ID=0
# Iterar sobre el CSV
for index, row in test_data.iterrows():
    filename = row['des_filename']
    img_path = os.path.join(image_dir, filename)

    # Comprovar si la imatge és vàlida
    if not os.path.exists(img_path) or os.path.getsize(img_path) == 0:
        print(img_path)
        continue

    # Carregar i preprocessar la imatge
    image = read_image(img_path).type(torch.float32)
    image_batch.append(image)
    filename_batch.append(filename)

    # Si hem acumulat un lot complet, fem prediccions
    if len(image_batch) == batch_size or index == len(test_data) - 1:
        # Combinar les imatges en un tensor i moure-les al dispositiu
        image_tensor = torch.stack(image_batch).to(device)

        # Prediccions
        preds = model_guillem(image_tensor)
        preds_np = preds.cpu().detach().numpy()
        pred_labels = np.argmax(preds_np, axis=1)

        # Assignar etiquetes i emmagatzemar resultats
        for i, fname in enumerate(filename_batch):
            tags = dataset.label_encoders.keys()
            d = {}

            for j, t in enumerate(tags):
                row_id = fname[:11]  # Extreure ID del nom de l'arxiu
                encoder = dataset.label_encoders[t]
                label_for_tag = pred_labels[i, j]
                try:
                    d[t] = encoder.inverse_transform([label_for_tag])[0]
                except:
                    d[t] = 'INVALID'

                row_data = {'test_id': row_id + '_' + t, 'des_value': d[t]}
                rows_output = pd.concat([rows_output, pd.DataFrame([row_data])], ignore_index=True)

        # Netejar el lot per processar el següent
        image_batch = []
        filename_batch = []
# Guardar les prediccions
rows_output.to_csv('MangoLoco-RandomGuess.csv', index=False)
